In [7]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [8]:
from itertools import combinations

# generate random combination with number n
def combine(temp_list, n):
    return list(combinations(temp_list, n))

In [9]:
# set semantics and compute strength of arguments

filename = "../../bags/td_qbaf.bag"
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG(filename)
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 9.499999999999982
Argument(name=s0, weight=0.5, strength=0.6983866081211344)
Argument(name=s1, weight=0.5, strength=0.6984075601349732)
Argument(name=s2, weight=0.5, strength=0.6983598317669508)
Argument(name=s3, weight=0.5, strength=0.6983598317669508)
Argument(name=s4, weight=0.5, strength=0.6983866081211344)
Argument(name=s5, weight=0.5, strength=0.6983561366841534)
Argument(name=s6, weight=0.5, strength=0.6983598317669508)
Argument(name=s7, weight=0.5, strength=0.6983561366841534)
Argument(name=s8, weight=0.5, strength=0.6983561366841534)
Argument(name=s9, weight=0.5, strength=0.500000816300687)
Argument(name=c0, weight=0.0, strength=0.8135102277027011)
Argument(name=c1, weight=0.0, strength=7.65430959223384e-06)
Argument(name=c2, weight=0.0, strength=0.8135536589680334)
Argument(name=c3, weight=0.0, strength=6.901377616601401e-06)
Argument(name=c4, weight=0.0, strength=0.8135150192670503)
Argument(name=c5, weight=0.0, strength=6.909952043440828e-06)

0.0009670739741814057

In [10]:
# get all the attacks and supports in QBAF
att_relation = model.BAG.get_attacks()
sup_relation = model.BAG.get_supports()
all_relation = att_relation + sup_relation

In [11]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', 'c0', 'c1'),
 ('att', 'c1', 'c0'),
 ('att', 'c2', 'c3'),
 ('att', 'c3', 'c2'),
 ('att', 'c4', 'c5'),
 ('att', 'c5', 'c4'),
 ('sup', 's0', 'c2'),
 ('sup', 's1', 'c1'),
 ('sup', 's1', 'c2'),
 ('sup', 's2', 'c4'),
 ('sup', 's3', 'c4'),
 ('sup', 's4', 'c2'),
 ('sup', 's5', 'c0'),
 ('sup', 's6', 'c4'),
 ('sup', 's7', 'c0'),
 ('sup', 's8', 'c0'),
 ('sup', 's9', 'c3'),
 ('sup', 's9', 'c5'),
 ('sup', 'c2', 's0'),
 ('sup', 'c1', 's1'),
 ('sup', 'c2', 's1'),
 ('sup', 'c4', 's2'),
 ('sup', 'c4', 's3'),
 ('sup', 'c2', 's4'),
 ('sup', 'c0', 's5'),
 ('sup', 'c4', 's6'),
 ('sup', 'c0', 's7'),
 ('sup', 'c0', 's8'),
 ('sup', 'c3', 's9'),
 ('sup', 'c5', 's9')]

In [12]:
# set topic argument
topic_a = "c0"
topic_arg = model.BAG.arguments[topic_a]

In [13]:
# compute strength for all arguments
for arg in model.BAG.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('s0', 0.5, 0.6983866081211344)
('s1', 0.5, 0.6984075601349732)
('s2', 0.5, 0.6983598317669508)
('s3', 0.5, 0.6983598317669508)
('s4', 0.5, 0.6983866081211344)
('s5', 0.5, 0.6983561366841534)
('s6', 0.5, 0.6983598317669508)
('s7', 0.5, 0.6983561366841534)
('s8', 0.5, 0.6983561366841534)
('s9', 0.5, 0.500000816300687)
('c0', 0.0, 0.8135102277027011)
('c1', 0.0, 7.65430959223384e-06)
('c2', 0.0, 0.8135536589680334)
('c3', 0.0, 6.901377616601401e-06)
('c4', 0.0, 0.8135150192670503)
('c5', 0.0, 6.909952043440828e-06)


In [ ]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    # split current and the rest relation
    cur_relation = all_relation_new[i]
    rest_relation = all_relation_new.copy()
    rest_relation.remove(cur_relation)

    # compute combinations for the rest_relation
    combine_rest_relation = []
    for j in range(len(rest_relation)+1):
        combine_rest_relation.extend(combine(rest_relation, j))

    difference = []
    coefficient = []

    # remove relations in combine_rest_relation
    for j in combine_rest_relation:
        for k in j:
            # remove att
            if k[0]=="att":
                temp_attacker = model.BAG.remove_attack(k[1],k[2]) # k[1] attacks k[2]
            # remove sup
            elif k[0]=="sup":
                temp_supporter = model.BAG.remove_support(k[1],k[2]) # k[1] supports k[2]

        # compute sigma(S U {i})
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength

        # remove cur_relation
        if cur_relation[0]=="att":
            temp_attacker = model.BAG.remove_attack(cur_relation[1],cur_relation[2])
        elif cur_relation[0]=="sup":
            temp_supporter = model.BAG.remove_support(cur_relation[1],cur_relation[2])

        # compute sigma(S)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)

        # compute the coefficient
        S = len(j)
        coefficient.append(math.factorial(n-1-S) * math.factorial(S) / math.factorial(n))

        # recover QBAF for the next iteration
        model = grad.semantics.QuadraticEnergyModel()
        model.approximator = grad.algorithms.RK4(model)
        model.BAG = grad.BAG(filename)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    # compute attributions
    attribution[cur_relation] = sum(np.multiply(coefficient, difference))

In [ ]:
attribution